In [1]:
import os
import duckdb
import logging
import pytz
from datetime import datetime, timedelta
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Setup date
date = datetime.now(pytz.timezone("Asia/Bangkok")) - timedelta(days=2)
day = date.strftime("%Y-%m-%d")
print(f'Processing date: {day}')

Processing date: 2025-03-30


In [3]:
# Load environment variables
load_dotenv()
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD")
MINIO_BUCKET = os.getenv("MINIO_BUCKET")

PG_HOST = os.getenv("POSTGRES_HOST")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PASSWORD = os.getenv("POSTGRES_PASSWORD")
PG_DB = os.getenv("POSTGRES_DB")

In [4]:
# Check Required Envs
required_envs = {
    "MINIO_ENDPOINT": MINIO_ENDPOINT,
    "MINIO_ROOT_USER": MINIO_ACCESS_KEY,
    "MINIO_ROOT_PASSWORD": MINIO_SECRET_KEY,
    "MINIO_BUCKET": MINIO_BUCKET,
    "POSTGRES_HOST": PG_HOST,
    "POSTGRES_PORT": PG_PORT,
    "POSTGRES_USER": PG_USER,
    "POSTGRES_PASSWORD": PG_PASSWORD,
    "POSTGRES_DB": PG_DB,
}
missing = [k for k, v in required_envs.items() if not v]
if missing:
    raise EnvironmentError(f"Missing required environment variables: {missing}")

In [5]:
# Setup Logging
log_dir = Path("/home/jovyan/work/data/logs")
log_dir.mkdir(parents=True, exist_ok=True)
log_path = log_dir / f"load_to_postgres_{day}.log"

logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

logging.info("Start loading data from MinIO to PostgreSQL")

In [6]:
# Connect to DuckDB and MinIO
parquet_pattern = f"s3://{MINIO_BUCKET}/{day}/flight_weather.parquet/*.parquet"
os.environ["AWS_ACCESS_KEY_ID"] = MINIO_ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = MINIO_SECRET_KEY

db = duckdb.connect()

db.execute("INSTALL httpfs;")
db.execute("LOAD httpfs;")
db.execute("SET s3_region='us-east-1';")
db.execute(f"SET s3_endpoint='{MINIO_ENDPOINT}';")
db.execute("SET s3_url_style='path';")
db.execute("SET s3_use_ssl=false;")

logging.info(f"Reading parquet from: {parquet_pattern}")

In [7]:
# Load Parquet to DuckDB
try:
    df = db.execute(f"SELECT * FROM '{parquet_pattern}'").fetchdf()
    logging.info(f"Loaded {len(df)} records from MinIO")
except Exception as e:
    logging.error(f"Failed to load parquet: {e}")
    raise

In [8]:
# Write to PostgreSQL
pg_conn_str = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"

try:
    db.execute("INSTALL postgres;")
    db.execute("LOAD postgres;")
    db.execute(f"ATTACH '{pg_conn_str}' AS postgres (TYPE postgres);")
    db.execute("""
        CREATE OR REPLACE TABLE postgres.public.flight_weather AS
        SELECT * FROM df;
    """)
    logging.info("Data successfully written to PostgreSQL")
except Exception as e:
    logging.error(f"Failed to write to PostgreSQL: {e}")
    raise

logging.info("Load to PostgreSQL completed successfully")